### 문제 4-2 : 조건부 분기가 있는 메뉴 추천 시스템 (LangGraph 사용)

In [1]:
import os
import re
import uuid
from typing import List, Literal
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain.agents import tool
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langgraph.graph import StateGraph, END
from langgraph.graph.message import MessagesState
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver

In [2]:
def setup_environment():
    load_dotenv()
    if not os.getenv("UPSTAGE_API_KEY"):
        raise ValueError("UPSTAGE_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")
    print("환경 설정이 완료되었습니다.")

In [3]:
def parse_menu_data(file_path: str) -> List[dict]:
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    menu_items = []
    items = re.split(r'\n(?=\d+\.\s)', content)
    for item in items:
        if not item.strip():
            continue
        
        name_match = re.search(r'^\d+\.\s*(.+)', item)
        price_match = re.search(r'•\s*가격:\s*(₩[\d,]+)', item)
        description_match = re.search(r'•\s*설명:\s*(.+)', item, re.DOTALL)
        
        if name_match and price_match and description_match:
            menu_items.append({
                "menu_name": name_match.group(1).strip(),
                "price": price_match.group(1).strip(),
                "description": description_match.group(1).strip()
            })
    return menu_items

In [4]:
def prepare_vector_db(file_path: str):
    menu_data = parse_menu_data(file_path)
    
    documents = [
        Document(
            page_content=f"메뉴명: {item['menu_name']}\n가격: {item['price']}\n설명: {item['description']}",
            metadata={"menu_name": item['menu_name']}
        ) for item in menu_data
    ]
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    db = FAISS.from_documents(documents, embeddings)
    print("카페 메뉴 Vector DB가 준비되었습니다.")
    return db

In [5]:
class GraphState(MessagesState):
    inquiry_type: Literal["menu", "price", "recommendation", "general"]
    context: str

In [6]:
file_path = '../data/cafe_menu_data.txt'

menu_db = prepare_vector_db(file_path)
llm = ChatUpstage(model="solar-1-mini-chat")

카페 메뉴 Vector DB가 준비되었습니다.


In [7]:
def classify_inquiry(state: GraphState) -> GraphState:
    print("--- 노드: 문의 유형 분류 ---")
    user_message = state["messages"][-1].content
    
    price_keywords = ["가격", "얼마", "비용"]
    recommend_keywords = ["추천", "어떤", "골라줘"]
    menu_keywords = ["메뉴", "뭐 있어", "종류"]

    if any(keyword in user_message for keyword in price_keywords):
        inquiry_type = "price"
    elif any(keyword in user_message for keyword in recommend_keywords):
        inquiry_type = "recommendation"
    elif any(keyword in user_message for keyword in menu_keywords):
        inquiry_type = "menu"
    else:
        # 키워드가 없으면 의미론적 검색을 위해 일단 'menu'로 분류
        inquiry_type = "menu"
        
    print(f"분류된 유형: {inquiry_type}")
    return {"inquiry_type": inquiry_type}

In [8]:
def search_menu_info(state: GraphState) -> GraphState:
    print(f"--- 노드: 메뉴 정보 검색 ({state['inquiry_type']}) ---")
    inquiry_type = state["inquiry_type"]
    user_message = state["messages"][-1].content

    if inquiry_type == "price":
        # 가격 문의 시, 더 넓은 범위에서 검색
        docs = menu_db.similarity_search(user_message, k=5)
    elif inquiry_type == "recommendation":
        # 추천 요청 시, 사용자 메시지를 기반으로 검색하고 결과가 없으면 인기 메뉴 검색
        docs = menu_db.similarity_search(user_message, k=3)
        if not docs:
            docs = menu_db.similarity_search("인기 메뉴", k=3)
    else: # "menu" 또는 일반
        docs = menu_db.similarity_search(user_message, k=4)
        
    context = "\n\n".join([doc.page_content for doc in docs])
    return {"context": context}

In [9]:
def generate_response(state: GraphState) -> GraphState:
    print("--- 노드: 답변 생성 ---")
    prompt = f"""당신은 친절한 카페 직원입니다. 아래 주어진 정보를 바탕으로 손님의 질문에 답변해주세요.

[검색된 메뉴 정보]
{state['context']}

[손님 질문]
{state['messages'][-1].content}

답변:
"""
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [10]:
def handle_general_inquiry(state: GraphState) -> GraphState:
    print("--- 노드: 일반 문의 처리 ---")
    response = AIMessage(content="죄송하지만, 저는 카페 메뉴에 대한 정보만 드릴 수 있어요. 다른 메뉴 관련 질문이 있으신가요?")
    return {"messages": [response]}

In [11]:
def route_inquiry(state: GraphState) -> Literal["search_info", "general_inquiry"]:

    user_message = state["messages"][-1].content
    docs = menu_db.similarity_search(user_message, k=1)
    
    if "메뉴" in user_message or "가격" in user_message or "추천" in user_message or docs:
        return "search_info"
    else:
        return "general_inquiry"


In [12]:
def create_graph():
    builder = StateGraph(GraphState)

    builder.add_node("classify", classify_inquiry)
    builder.add_node("search_info", search_menu_info)
    builder.add_node("generate_response", generate_response)
    builder.add_node("general_inquiry", handle_general_inquiry)

    builder.set_entry_point("classify")
    
    builder.add_conditional_edges(
        "classify",
        route_inquiry,
        {
            "search_info": "search_info",
            "general_inquiry": "general_inquiry"
        }
    )
    
    builder.add_edge("search_info", "generate_response")
    builder.add_edge("generate_response", END)
    builder.add_edge("general_inquiry", END)
    
    graph = builder.compile(checkpointer=MemorySaver())
    print("LangGraph 그래프가 생성되었습니다.")
    return graph

In [13]:
def run_simulation(graph):
    print("\n--- 카페 메뉴 추천 챗봇 시뮬레이션 시작 ---")
    
    # 각 대화 세션을 식별하기 위한 고유 ID
    thread_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": thread_id}}

    questions = [
        "안녕하세요! 어떤 메뉴들이 있나요?",
        "아메리카노 가격은 얼마인가요?",
        "달콤한 디저트 좀 추천해주세요.",
        "가장 인기 있는 메뉴는 뭐야?",
        "오늘 날씨 어때요?"
    ]

    for question in questions:
        print(f"\nHuman: {question}")
        
        # 이전 대화 기록이 자동으로 checkpointer에 의해 관리됨
        result = graph.invoke({"messages": [HumanMessage(content=question)]}, config=config)
        
        response = result["messages"][-1]
        print(f"AI: {response.content}")

In [14]:
if __name__ == "__main__":
    setup_environment()
    chatbot_graph = create_graph()
    run_simulation(chatbot_graph)

환경 설정이 완료되었습니다.
LangGraph 그래프가 생성되었습니다.

--- 카페 메뉴 추천 챗봇 시뮬레이션 시작 ---

Human: 안녕하세요! 어떤 메뉴들이 있나요?
--- 노드: 문의 유형 분류 ---
분류된 유형: recommendation
--- 노드: 메뉴 정보 검색 (recommendation) ---
--- 노드: 답변 생성 ---
AI: 안녕하세요! 저희 카페에서는 다음과 같은 메뉴를 제공하고 있습니다:

1. **티라미수** (₩7,500)  
   - 이탈리아 전통 디저트로, 마스카포네 치즈와 에스프레소에 적신 레이디핑거를 층층이 쌓아 만들었습니다. 부드럽고 달콤한 맛이 특징이며, 코코아 파우더로 마무리하여 깊은 풍미를 자랑합니다.

2. **카푸치노** (₩5,000)  
   - 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 일품이며, 계피 파우더를 뿌려 제공합니다.

3. **콜드브루** (₩5,000)  
   - 찬물에 12-24시간 우려낸 콜드브루 원액을 사용한 시원한 커피입니다. 부드럽고 달콤한 맛이 특징이며, 산미가 적어 누구나 부담 없이 즐길 수 있습니다. 얼음과 함께 시원하게 제공됩니다.

이 중에서 어떤 메뉴를 주문하시겠어요? 추가로 궁금한 사항이 있으시면 언제든지 물어보세요!

Human: 아메리카노 가격은 얼마인가요?
--- 노드: 문의 유형 분류 ---
분류된 유형: price
--- 노드: 메뉴 정보 검색 (price) ---
--- 노드: 답변 생성 ---
AI: 아메리카노의 가격은 ₩4,500입니다. 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피로, 원두 본연의 맛과 깔끔하고 깊은 풍미를 느끼실 수 있습니다. 설탕이나 시럽을 추가하셔서 더욱 맛있게 즐기실 수 있습니다.

Human: 달콤한 디저트 좀 추천해주세요.
--- 노드: 문의 유형 분류 ---
분류된 유형: recommendation
--- 노드: